In [110]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para la creación de modelo Ridge
from sklearn.linear_model import Ridge
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline


In [111]:
data = pd.read_excel("venta_inmuebles_data.xlsx")

In [112]:
# Cantidad de datos y número de variables
data.shape

(5466, 10)

In [113]:
data.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción,precio
0,3,1,1180,5650,1,3,7,0,1955,221900
1,2,1,770,10000,1,3,6,0,1933,180000
2,4,3,1960,5000,1,5,7,910,1965,604000
3,3,2,1680,8080,1,3,8,0,1987,510000
4,3,1,1780,7470,1,3,7,730,1960,229500


In [114]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()


In [115]:
# El tamaño de los datos preparados
data_t.shape

(5465, 10)

In [116]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['precio']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['precio'], axis=1)

In [117]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [118]:
#Como ya has podido conocer la regresión Ridge utiliza la norma L2 del vector de coeficientes como término de regularización. 
#Para controlar la cantidad de regularización se utiliza el hiperparámetro alpha. Probemos con un valor igual a 2.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=2))
modelo_ridge   

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=2))])

In [119]:
# Ajuste del modelo
modelo_ridge.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=2))])

In [120]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120680.46
MAE: 95100.03
R²: 0.54


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Pero ¿Cómo saber el mejor valor de alpha? Este hiperparámetro hay que ajustarlo a los datos y no podemos utilizar el conjunto test para realizar esta tarea. Debemos entonces recurrir a las técnicas de validación las cuales, en general, puedes implementarlas de dos maneras: utilizando un conjunto de validación o empleando la validación cruzada de k particiones (k-fold cross validation).

a) Con un conjunto de validación. Si utilizas esta vía el conjunto de entrenamiento hay que dividirlo en dos: uno para la construcción del modelo (sería el nuevo conjunto de entrenamiento) y el otro para la validación (para determinar los valores de los hiperparámetros y complejidad). 

Veamos cómo hacerlo.


In [121]:
# Dividimos el conjunto de entrenamiento en dos: una para la construcción del modelo (sería el nuevo conjunto de 
# entrenamiento) y otro para la validación, el cuál será utilizado para determinar el rendimiento del modelo con una
# combinación específica de hiperparámetros.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size = 0.15, random_state = 0)

In [122]:
## Fijemos un valor de alpha en 1, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=1))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94542.90


In [123]:
## Fijemos un valor de alpha en 2, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=2))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94541.85


In [124]:
## Fijemos un valor de alpha en 5, construyamos el modelo y probemos sobre validación. Utilicemos MAE como métrica de
# rendimiento para hacer la selección.
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=5))
modelo_ridge.fit(X_trainval,Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 94538.71


El mejor alpha es 5 

In [125]:
modelo_ridge = make_pipeline(StandardScaler(), Ridge(alpha=5))
modelo_ridge.fit(X_train,Y_train)
y_pred = modelo_ridge.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 120672.14
MAE: 95098.73
R²: 0.54


Ahora utilicemos la otra opción.

b) Validación cruzada de k-particiones. En este caso el conjunto de entrenamiento se divide en k particiones disjuntas. Luego, se fija un valor de alpha y se toma como conjunto de validación la primera partición y el resto para entrenar el modelo. Este proceso se repite para cada partición. Al finalizar, se determina el valor promedio de las métricas seleccionadas. Este ciclo se repite sobre diferentes valores de alpha y se seleccionan aquel que ofrezca el mejor rendimiento. 

Scikit-learn ofrece algunos métodos que automatizan el proceso de buscar los valores de los hiperparámetros. Uno de ellos es GridSearchCV, el cual se basa en la validación cruzada de k-particiones. Veamos cómo utilizarlo.

In [126]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [127]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'ridge__alpha': [1,2,5]}

In [128]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_ridge = Pipeline([
    ('scaler', StandardScaler()),  # Normalización
    ('ridge', Ridge())  # Modelo Ridge
])

In [129]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             n_jobs=-1, param_grid={'ridge__alpha': [1, 2, 5]})

In [130]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'ridge__alpha': 5}


In [131]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1)
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'ridge__alpha': 1}


In [132]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 120683.27
MAE: 95100.47
R²: 0.54


c:\Users\harve\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [133]:
# Obtener los coeficientes del modelo final
coeficientes = modelo_final.named_steps['ridge'].coef_
intercepto = modelo_final.named_steps['ridge'].intercept_
mejor_alpha = mejor_modelo.best_params_['ridge__alpha']

# Mostrar resultados
print("Intercepto:", intercepto)
print("Coeficientes:", coeficientes)
print("Mejor valor de alpha:", mejor_alpha)


Intercepto: 388661.6710887468
Coeficientes: [-15789.33065637  23078.74334749  54209.15280122   3199.79866556
  19041.89675594   7423.50497158  82305.02644344   4850.29194013
 -67719.77509985]
Mejor valor de alpha: 1
